# set-up

## Imports

In [ ]:
import datetime
import fnmatch
import matplotlib.cbook
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
import numpy as np
import os
import glob
import pandas as pd
import pickle
import platform
from pylab import rcParams
import scipy
from scipy import stats
import warnings

warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
startTimeNotebook = datetime.datetime.now()
if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/HeathenToaster/code
    %cd code

# session lists in a .py file
# import sessionlists
%run sessionlists
from VIGOR_utils import *
from VIGOR_MODELS_Functions import *

plt.style.use('./Figures/test.mplstyle')

## Define data path.

In [ ]:
#### Define folder with data 
if platform.system()=='Linux':
    root="/home/david/Desktop/ALLDATA"
    savePath="/home/david/Desktop/Save"
elif platform.system()=='Darwin':
    root="/Users/tom/Desktop/sequencesALLDATA"
    savePath="/Users/tom/Desktop/Save"
if 'COLAB_GPU' in os.environ:
    !gdown --id 1BSepSzm1-KQJlUvv8C23p_qyHvtzvrZ8
    !unzip -qq /content/code/sequencesALLDATA.zip
    root="/content/code/sequencesALLDATA"
    savePath="/content/Save"
    print("I'm running on Colab")
print("Path to data is: %s"%root)

retval = os.getcwd()
print("Current working directory: %s" % retval)
print("Save Path: ", savePath)

print(f'Found {len(glob.glob(root+"/*"))} rats in the data folder')

## Load processed data

In [ ]:
rat_markers = {}
sequence = {}

# define colors for each rat
palette = {'RatF00': (0.4, 0.0, 0.0), 'RatF01': (0.55, 0.13, 0.13), 'RatF02': (0.8, 0.2, 0.2),
           'RatM00': (0.0, 0.4, 0.0), 'RatM01': (0.13, 0.55, 0.13), 'RatM02': (0.2, 0.8, 0.2),
           'RatF20': (0.4, 0.0, 0.0), 'RatF21': (0.55, 0.13, 0.13), 'RatF22': (0.8, 0.2, 0.2),
           'RatM20': (0.0, 0.4, 0.0), 'RatM21': (0.13, 0.55, 0.13), 'RatM22': (0.2, 0.8, 0.2),
           'RatF30': (0.4, 0.0, 0.0), 'RatF31': (0.55, 0.13, 0.13), 'RatF32': (0.8, 0.2, 0.2), 'RatF33': (0.8, 0.2, 0.2),
           'RatM30': (0.0, 0.4, 0.0), 'RatM31': (0.13, 0.55, 0.13), 'RatM32': (0.2, 0.8, 0.2), 
           'RatF30L': (0.4, 0.0, 0.0), 'RatF31L': (0.55, 0.13, 0.13), 'RatF32L': (0.8, 0.2, 0.2), 'RatF33L': (0.8, 0.2, 0.2),
           'RatM30L': (0.0, 0.4, 0.0), 'RatM31L': (0.13, 0.55, 0.13), 'RatM32L': (0.2, 0.8, 0.2)}

# define brain status (lesion/CNO/intact) for each rat, used in plots
# needs to be properly implemented, setting is in behav_params for each session.
brainstatus = {'RatF00': 'normal', 'RatF01': 'normal', 'RatF02': 'normal',
               'RatM00': 'normal', 'RatM01': 'normal', 'RatM02': 'normal',
               'RatF20': 'thcre', 'RatF21': 'thcre', 'RatF22': 'thcre',
               'RatM20': 'thcre', 'RatM21': 'thcre', 'RatM22': 'thcre',
               'RatF30': 'lesion', 'RatF31': 'lesion', 'RatF32': 'normal', 'RatF33': 'normal',
               'RatM30': 'lesion', 'RatM31': 'normal', 'RatM32': 'normal',
                'RatF30L': 'lesion', 'RatF31L': 'lesion', 'RatF32L': 'biglesion', 'RatF33L': 'biglesion',
                'RatM30L': 'lesion', 'RatM31L': 'biglesion', 'RatM32L': 'biglesion'
               }

markers = {'normal': 'o', 'thcre': 'd', 'lesion': 'x', 'biglesion': 'X'}
lines = {'normal': '-', 'thcre': '--', 'lesion': ':', 'biglesion': '-.'}

# define list of rats to be analyzed
# first batch  
# animalList = ['RatF00', 'RatF01', 'RatF02', 'RatM00', 'RatM01', 'RatM02']

# all rats
# animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
# animalList += ['RatF30L', 'RatF31L', 'RatF32L', 'RatF33L', 'RatM30L', 'RatM31L', 'RatM32L']

# INTACT RATS
animalList = ['RatF00', 'RatF01', 'RatF02', 'RatM00', 'RatM01', 'RatM02', 'RatF32', 'RatF33', 'RatM31', 'RatM32']
# animalList += ['RatF20', 'RatF21', 'RatF22', 'RatM20', 'RatM21', 'RatM22']


for index, animal in enumerate(animalList):
    print(f'Loading data for {animal}')
    # define marker and color for each rat, used in plots
    if fnmatch.fnmatch(animal, 'RatF*'):
        rat_markers[animal]=[palette[animal], markers[brainstatus[animal]], lines[brainstatus[animal]]]
    elif fnmatch.fnmatch(animal, 'RatM*'):
        rat_markers[animal]=[palette[animal], markers[brainstatus[animal]], lines[brainstatus[animal]]]
    elif fnmatch.fnmatch(animal, 'Rat00*'):
        rat_markers[animal]=[(0.0, 0.0, 0.0), "$\u2426$",]
    else:
        print("error, this is not a rat you got here")

    # loop over all sessions for each rat and get the pickled preprocessed data. Data is processed in VIGOR_Preprocess.py
    # data (list or list of lists) for each variable is stored in a dictionary with keys (animal, session)
    for session in sorted(matchsession(animal, dist60+dist90+dist120 + TM20+TM10+TM2+TMrev2+TMrev10+TMrev20)):
                                                #  dist60bis+dist90bis+dist120bis)):

        # get the preprocessed data from the pickle file
        # In this notebook we only need the sequence of events, so we only load that
        biglesion = True if 'L' in session else False
        sequence[animal, session] = get_from_pickle(root, animal[0:6], session, name="sequence.p", biglesion=biglesion)

data60 = prepare_data_running_times(sequence, animalList, dist60)
data90 = prepare_data_running_times(sequence, animalList, dist90)
data120 = prepare_data_running_times(sequence, animalList, dist120)

data20 = prepare_data_running_times(sequence, animalList, TM20)
data10 = prepare_data_running_times(sequence, animalList, TM10)
data2 = prepare_data_running_times(sequence, animalList, TM2+TMrev2)
datarev10 = prepare_data_running_times(sequence, animalList, TMrev10)
datarev20 = prepare_data_running_times(sequence, animalList, TMrev20)

dataAll = prepare_data_running_times(sequence, animalList, dist60+dist90+dist120 + TM20+TM10+TM2+TMrev2+TMrev10+TMrev20)

# Introduction

## Task  
The task and the main behavioral results are presented in notebook 1.  
We developed an automated foraging task in which water restricted rats have to run back and
forth on a treadmill to obtain drops of water. Within one hour-long session, the
probability of getting a reward was alternatively high and low in 5 min-long uncued
blocks (dark and light gray in fig.1).  

<img src="Figures/Picturetask1.png" alt="task" width="300"/>  

## Experimental conditions
Across sessions, we manipulated the effort rats had to produce by either
modifying the length of the treadmill while its speed remained null, or by
manipulating the speed and direction of the belt to facilitate or counteract the
animals' crossings.

<img src="Figures/conditions.png" alt="conditions" width="600"/>

# Aim of the Notebook 
One of the variables that we analyze is the duration of the running times (blue part on the track of the animal, Fig 1A.).  
A quick analysis of the median running time shows that it increases along the 
session, but no clear modulation in function of reward (Fig1B.); i.e. animals are less motivated/more tired as time passes, they run slower and don't modulate their speed according to the reward probability.   

In this notebook we characterise how time and reward probability affects running time.  

Fig 1.  
A) Position of an example animal across a 120 cm session. Blue is run epoch, orange is running epoch.  
Reward probability in 5 min blocks is either high (90%, dark gray), or low (10%, light gray)  
B) Median running time across session and blocks.  
Reward probability in 5 min blocks is either high (90%, dark gray), or low (10%, light gray)  

In [ ]:
# Fig 1.
animal, session = 'RatM01', 'RatM01_2021_07_22_17_14_48'  # 'RatF00', 'RatF00_2021_07_24_15_28_05'
fig, axs = plt.subplots(1, 2, figsize = (24, 4), gridspec_kw={'width_ratios': [5, 1]})

# Load preprocessed data for one animal and one session. params contains the parameters of the session,
# runningTimeInLeftBin and runningTimeInRightBin contain the running time in each time bin for each animal
example_params = get_from_pickle(root, animal, session, name="params.p")
example_runningTimeInLeftBin, example_runningTimeInRightBin = get_from_pickle(root, animal, session, name="timeRun.p")

# plot the trajectory
plot_animal_trajectory(root=root, animal=animal, session=session, params=example_params, barplotaxes=[0, 3600, 0, 120], 
                 xyLabels=["Time (min)", "Position along treadmill (cm)"], title="Position of example animal across a 120 cm session", ax=axs[0])

# plot the median running time in each time block
plot_median_per_bin([example_runningTimeInLeftBin[i]+example_runningTimeInRightBin[i] for i in range(0, 12)], 
                example_params['rewardProbaBlock'], example_params['blocks'], barplotaxes=[0, 3600/60, 0, 2.5], 
                color=['dodgerblue'], xyLabels=["Time (min)","running time (s)"], title="Median running time", scatter=False, stat='Med. ', ax=axs[1]);

# Data binning

The data is binned as in the running time model. 
This yields the following complete distribution.

In [ ]:
# Fig 3.
# distribution of running times

animal = 'RatM01'
exampledata = prepare_data_running_times(sequence, animalList, dist120)
plot_full_distribution_run(exampledata, animal, plot_fit=True)

# Distributions

## Cauchy distribution

We use the Cauchy distribution to fit the running times.  
Why --> distribution has big peak, and some running times are long
Cauchy looks like normal distribution but has a bigger peak and a fat tail. Fits well with only two parameters and I did not find anything better (lognormal, frechet, norm, weibull have been tested). Is the ratio of two independant normally distributed variables (e.g. Speed = Thirst/Tiredness ??).  

The probability density function of the Cauchy distribution is:  

$Cauchy(x|\mu, \sigma) = \frac{1}{\pi \sigma \left[1+ \left(\frac{x-\mu}{\sigma}\right)²\right]}$  

The cauchy distribution has 2 parameters: $\mu$, $\sigma$.  
- $\mu$ is location of the peak (shifts the distribution)  
- $\sigma$ is half the width at half the peak (spread).  $\sigma > 0$

Note that the Cauchy distribution classically uses the parameters $x_0$ and $\gamma$ (because mean and variance are undefined), we use $\mu$ instead of $x_0$ and $\sigma$ instead of $\gamma$ to avoid confusion with parameters in the idle time model.  


&nbsp;  
A) Changes in the Cauchy distribution as a result of changes in the $\mu$, $\sigma$ parameters.  


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
x = np.linspace(-5, 50, 1000)
axs[0].plot(x, stats.cauchy.pdf(x, 0, 1), 'k-', label='Default')
axs[0].plot(x, stats.cauchy.pdf(x, 2.5, 1), 'c', label='increased mu')
axs[0].plot(x, stats.cauchy.pdf(x, 0, 2.5), 'r-', label='increased sigma')
axs[0].plot(x, stats.cauchy.pdf(x, 1.35, .2), '--', label='expe data example')
axs[0].set_ylabel('PDF')
axs[0].set_xlabel('t')
axs[0].set_xlim(-5, 5)
axs[0].set_ylim(0, 1)
axs[0].legend()

axs[1].plot(x, stats.cauchy.sf(x, 0, 1), 'k-', label='Default')
axs[1].plot(x, stats.cauchy.sf(x, 2.5, 1), 'c', label='increased mu')
axs[1].plot(x, stats.cauchy.sf(x, 0, 2.5), 'r-', label='increased sigma')
axs[1].plot(x, stats.cauchy.sf(x, 1.35, .2), '--', label='expe data example')
axs[1].set_xlabel('t')
axs[1].set_xlim(0.01, 50)
axs[1].set_ylim(0.01, 1.1)
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[1].set_ylabel('CDF')

## Fitting
We fit running time distributions $x$ to the Cauchy distribution using the maximum likelihood estimation method.

We find the parameters $\hat{\mu}, \hat{\sigma}$ that maximizes the log-likelihood function $\ell (Cauchy(x); \mu, \sigma)$, with $\mu, \sigma \in \mathrm{M}, \Sigma$, using the default 'L-BFGS-B' method from scipy. https://en.wikipedia.org/wiki/Limited-memory_BFGS#L-BFGS-B  
 
Fig 5.
Sanity check: We generate synthetic running times from a Wald distribution with known $\mu, \sigma$ parameters, and fit them to check that the fitted parameters are the same as the known parameters. We are able to recover the parameters well (QUANTIFY).  

TOP LEFT) Synthetic data fit with different values of $\sigma$.  
TOP RIGHT) Synthetic data fit with different values of $\mu$  
BOT LEFT&RIGHT) Hidden (lines) and recovered (dots) parameters

In [ ]:
N = 1000
std = .1
MEAN = 1

fig, axs = plt.subplots(2, 2, figsize=(10, 10))
x = np.linspace(0.01, 500, 1000)
means = np.linspace(1, 2, 10)
plot_color_line(axs[1, 0], means, means, means, cmap = 'autumn', vmin=0, vmax=2.5, alpha=1, linewidth=2, linestyle = '-', zorder = 1)
axs[1, 0].plot(means, [std for _ in means], color='c')
for mean in means:
    mu, sigma, _ = example_cauchy_fit(mean, std, N, ax=axs[0, 0], color=plt.get_cmap('autumn')(mean / max(means)))
    axs[1, 0].scatter(mean, sigma, color='c', label='')
    axs[1, 0].scatter(mean, mu, color=plt.get_cmap('autumn')(mean / max(means)), label='mu_fit')
    axs[1, 0].set_xlabel('sigma ground truth')
    axs[1, 0].set_ylabel('fitted params')

As = np.linspace(.05, .15, 10)
plot_color_line(axs[1, 1], As, As, As, cmap = 'winter', vmin=0, vmax=.25, alpha=1, linewidth=2, linestyle = '-', zorder = 1)
axs[1, 1].plot(As, [MEAN for _ in As], color='r')
for A in As:
    mu, sigma, _ = example_cauchy_fit(MEAN, A, N, ax=axs[0, 1], color=plt.get_cmap('winter')(A / max(As)))
    axs[1, 1].scatter(A, sigma, color=plt.get_cmap('winter')(A / max(As)), label='sigma_fit')
    axs[1, 1].scatter(A, mu, color='r', label='')
    axs[1, 1].set_xlabel(' mu ground truth')
    axs[1, 1].set_ylabel('fitted params')

legend_without_duplicate_labels(axs[1, 0])
legend_without_duplicate_labels(axs[1, 1])

# Running time model


We have the Cauchy distribution with $\mu$ and $\sigma$ parameters that we can fit and recover from individual running times distributions. 
We've seen earlier that the sub-distributions of running times in our data changes with time $t$ but not with the reward history $R$.  
Can we link $t$ and $R$ to $\mu$ and/or $\sigma$?  

To evaluate how $\mu$ and $\sigma$ change with $t$ and/or $R$ we define the following model:

$\mathrm{M}(t, R) = \mu_0 + \mu_t t + \mu_R R$  
$\Sigma(t, R) = \sigma_0 + \sigma_t t + \sigma_R R$  

With:  
$\mu_0, \sigma_0$ parameters at the beginning of the session (0-10 min) and $\frac{3}{3}$ rewards have been obtained  
$\mu_t, \sigma_t$ how $\mu$ and $\sigma$ evolve linearly with time  
$\mu_R, \sigma_R$ how $\mu$ and $\sigma$ evolve linearly with reward history  


We find the best $\mu_0, \mu_t, \mu_R, \sigma_0, \sigma_t, \sigma_R$ that minimize the total error of the model using maximum likelihood estimation.  

The total error of the model is same method as idle time


## Graphical representation of $\mu$ and $\sigma$ evolution
Evolution of $\mathrm{A}$ and $\sigma$ on the z axis, with $R$ on the x axis, $t$ on the y axis.  
- $\mathrm{M}$: starts low, increases with $t$, no change with $R$  
- $\sigma$: 

In [ ]:
# NOT THE SAME ANIMAL AS BEFORE (RatM01)!!!!!!!!

# example fit model on all 120 cm session for each animal
# print fitted parameters and loss for each animal
animal = 'RatF01'
exampledata = prepare_data_running_times(sequence, animalList, dist120)
p, loss = modelrun_fit(exampledata[animal])
(mu, sigma, mu_t, sigma_t, mu_R, sigma_R) = p

print(f"{animal}  mu:{mu:.2f}, mut:{mu_t:.2f}, muR: {mu_R:.2f}, || s: {sigma:.2f}, st: {sigma_t:.2f}, sR: {sigma_R:.2f} ||| loss: {loss:.2f}")

fig, axs = plt.subplots(1, 2, figsize=(10, 5), subplot_kw={'projection': '3d'})
plot_parameter_evolutionRun((mu, sigma, mu_t, sigma_t, mu_R, sigma_R), axs=axs)

In [ ]:
targetlist = generate_targetList(seq_len=4)[::-1]
fig, axs = plt.subplots(1, 3, figsize=(21, 7))

the_keys_i_want = [(False, False, False, False), 
                    (True, False, False, False), 
                    (False, True, False, False), 
                    (False, False, True, False), 
                    (False, False, False, True), 
                    (True, True, True, True)]

if os.path.exists("picklejar/running_time_model_BIC_ablation.p"):
    losses = pickle.load(open("picklejar/running_time_model_BIC_ablation.p", "rb"))
else:
    losses = {}
    for animal in animalList:
        losses[animal] = {}
        for mu_t_fixed in [False, True]:
            for sigma_t_fixed in [False, True]:
                for mu_R_fixed in [False, True]:
                    for sigma_R_fixed in [False, True]:
                        number_of_params = 2  # mu, sigma
                        number_of_extra_params = mu_t_fixed+sigma_t_fixed+mu_R_fixed+sigma_R_fixed
                        loss = modelrun_fit(dataAll[animal],
                                            f=modelrun_compare,
                                            mu_t_fixed=mu_t_fixed, 
                                            sigma_t_fixed=sigma_t_fixed, 
                                            mu_R_fixed=mu_R_fixed, 
                                            sigma_R_fixed=sigma_R_fixed, 
                                            N_params=number_of_params + (4 - number_of_extra_params),
                                            )[1]

                        losses[animal][(mu_t_fixed, sigma_t_fixed, mu_R_fixed, sigma_R_fixed)] = loss

for animal in animalList:
    axs[0].scatter(np.arange(16), losses[animal].values(), color=rat_markers[animal][0], label=animal)
    axs[0].set_title(f"loss")
    axs[0].set_ylabel(r'$\Sigma$ Loss')
    axs[0].set_xticks(np.arange(-1, 16))
    axs[0].set_xticklabels(dict_to_xticklabels(losses[animal], labels=['μt', 'σt', 'μR', 'σR']))
    # axs[0].set_ylim([80000, 140000])
    axs[0].set_xlim([-1.5, 16.5])

    axs[1].scatter(np.arange(16), list(losses[animal].values())/losses[animal][False, False, False, False], color=rat_markers[animal][0], label=animal)
    axs[1].set_title(f"nomalized loss")
    axs[1].set_ylabel(r'$\Sigma$ Loss, Normalized')
    axs[1].set_xticks(np.arange(-1, 16))
    axs[1].set_xticklabels(dict_to_xticklabels(losses[animal], labels=['μt', 'σt', 'μR', 'σR']))
    # axs[1].set_ylim([.975, 1.5])
    axs[1].set_xlim([-1.5, 16.5])
    axs[1].axhline(1, color='k', linestyle='--')

    for i, key in enumerate(the_keys_i_want):
        axs[2].scatter(i+1, losses[animal][key]/losses[animal][False, False, False, False], color=rat_markers[animal][0], label=animal)
    axs[2].set_title(f"normalized loss (1)")
    axs[2].set_ylabel(r'$\Sigma$ Loss')
    axs[2].set_xticks(np.arange(7))
    axs[2].set_xticklabels(dict_to_xticklabels({k:losses[animal][k] for k in the_keys_i_want}, labels=['μt', 'σt', 'μR', 'σR']))
    # axs[2].set_ylim([.975, 1.5])
    axs[2].set_xlim([-.5, 6.5])
    axs[2].axhline(1, color='k', linestyle='--')

# average by key
means = [np.mean([losses[animal][key] for animal in animalList]) for key in losses[animal].keys()]
yerr = [np.std([losses[animal][key] for animal in animalList]) for key in losses[animal].keys()]
axs[0].errorbar(np.arange(16)+.25, means, yerr=yerr, color='k', label='Mean', marker='o', fmt=' ')

means_norm = [np.mean([losses[animal][key]/losses[animal][False, False, False, False] for animal in animalList]) for key in losses[animal].keys()]
yerr_norm = [np.std([losses[animal][key]/losses[animal][False, False, False, False] for animal in animalList]) for key in losses[animal].keys()]
axs[1].errorbar(np.arange(16)+.25, means_norm, yerr=yerr_norm, color='k', label='Mean', marker='o', fmt=' ')

means_norm_select = [np.mean([losses[animal][key]/losses[animal][False, False, False, False] for animal in animalList]) for key in the_keys_i_want]
yerr_norm_select = [np.std([losses[animal][key]/losses[animal][False, False, False, False] for animal in animalList]) for key in the_keys_i_want]
axs[2].errorbar(np.arange(len(the_keys_i_want))+1.25, means_norm_select, yerr=yerr_norm_select, color='k', label='Mean', marker='o', fmt=' ')

_losses1 = [losses[animal][False, False, False, False]/losses[animal][False, False, False, False] for animal in animalList]
_losses2 = [losses[animal][True, False, False, False]/losses[animal][False, False, False, False] for animal in animalList]
_losses3 = [losses[animal][False, True, False, False]/losses[animal][False, False, False, False] for animal in animalList]
_losses4 = [losses[animal][False, False, True, False]/losses[animal][False, False, False, False] for animal in animalList]
_losses5 = [losses[animal][False, False, False, True]/losses[animal][False, False, False, False] for animal in animalList]
_losses6 = [losses[animal][True, True, True, True]/losses[animal][False, False, False, False] for animal in animalList]

print("Effect of parameters on loss (Friedman)")
f_test = stats.friedmanchisquare(_losses1, _losses2, _losses3, _losses4, _losses5, _losses6)
print(f'Friedman test: F={f_test[0]:.3f}, p={f_test[1]:.3f}')
print(" ")
print("comparisons: (wilcoxon)")
test_all_keys_between_themselves(losses, the_keys_i_want, axs[2])

# pickle.dump(losses, open("picklejar/running_time_model_BIC_ablation.p", "wb"))

# Results
### Example animal fit - 120cm
Fit of the model for the same example animal on all Dist-120cm sessions.  

In [ ]:
animal = 'RatM01'
exampledata = prepare_data_running_times(sequence, animalList, dist120)
plot_full_distribution_run(exampledata, animal, plot_fit=True)

### All experimental conditions fit
We can now do the same as above for all experimental conditions (Distances and TM speed) and for each rat.  
Each color is a different rat.  

$\mathrm{M}$:  
- $\mu_0$ > 0, positive bound at start, increases with distance and when TM speed reverse  
- $\mu_t$ > 0, $\mu$ increases with time  
- $\mu_R$ = 0, no effect of reward history on $\mu$  

The average running time increases when distance increases, and when TM is reversed -> animals do not manage to keep up the pace. 
The average running time increases with time, but not with reward history.  
**$\mu$ is congruent with *tiredness*, low at the start of session, increases with time/total distance traveled**


$\sigma$:  
- $\sigma_0$ > 0, initial variability  
- $\sigma t$ ~ 0, small effect of time on $\sigma$  
- $\sigma R$ ~ 0, small effect of reward history on $\sigma$  

**Variability increases slightly with time and when reward is low**  
 

**Stats**
Not normal, so use wilcoxon to see if mean in each condition is != 0.  
Wilcoxon to see if difference between two conditions. 

In [ ]:
fig, axs = plt.subplots(6, 3, figsize=(12, 20), gridspec_kw={'width_ratios': [5, 5, 2]})
ylabels = [r'$\mu_0$', r"$\mu_t$", r"$\mu_R$", r'$\sigma_0$', r"$\sigma_t$", r"$\sigma_R$"]
ylims = [[-.2, 2.5], [-.05, .2], [-.15, .15], [-.05, .2], [-.05, .05], [-.025, .1], ]
mu_t_fixed, sigma_t_fixed, mu_R_fixed, sigma_R_fixed = False, False, False, False
# mu_t_fixed, sigma_t_fixed, mu_R_fixed, sigma_R_fixed = False, True, True, False
# mu_t_fixed, sigma_t_fixed, mu_R_fixed, sigma_R_fixed = False, False, True, False
# mu_t_fixed, sigma_t_fixed, mu_R_fixed, sigma_R_fixed = True, True, True, True

mu, mu_t, mu_R, sigma, sigma_t, sigma_R, loss = {}, {}, {}, {}, {}, {}, {}
if os.path.exists("picklejar/running_time_model_parameters_fit.p"):
    mu, mu_t, mu_R, sigma, sigma_t, sigma_R, loss = pickle.load(open("picklejar/running_time_model_parameters_fit.p", "rb"))
else:
    for animal in animalList:
        mu[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}
        sigma[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}
        mu_t[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}
        sigma_t[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}
        mu_R[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}
        sigma_R[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}
        loss[animal] = {"60": 0, "90": 0, "120": 0, "20": 0, "10": 0, "2": 0, "rev10": 0, "rev20": 0}

        for cond, data in zip(["60", "90", "120", "20", "10", "2", "rev10", "rev20"], [data60, data90, data120, data20, data10, data2, datarev10, datarev20]):
            (mu[animal][cond], sigma[animal][cond], \
            mu_t[animal][cond], sigma_t[animal][cond], \
            mu_R[animal][cond], sigma_R[animal][cond]), loss[animal][cond] = modelrun_fit(data[animal], mu_t_fixed=mu_t_fixed, 
                                                                                                                                        sigma_t_fixed=sigma_t_fixed, 
                                                                                                                                        mu_R_fixed=mu_R_fixed, 
                                                                                                                                        sigma_R_fixed=sigma_R_fixed, 
                                                                                                                                        )
vars = [mu, mu_t, mu_R, sigma, sigma_t, sigma_R, ]
for animal in animalList:
    for i, (var, ylabel, ylim) in enumerate(zip(vars, ylabels, ylims)):
        axs[i, 0].scatter(np.arange(3), [var[animal]["60"], var[animal]["90"], var[animal]["120"]], color=rat_markers[animal][0], label=animal, marker=rat_markers[animal][1])
        axs[i, 0].set_title(f"")
        axs[i, 0].set_xticks(np.arange(3))
        axs[i, 0].set_xticklabels(["60", "90", "120"])
        axs[i, 0].set_ylabel(ylabel)
        axs[i, 0].set_xlim(-.5, 2.5)
        axs[i, 0].set_ylim(ylim)
        axs[i, 0].axhline(0, color='black', linestyle='--', linewidth=1)

    for i, (var, ylabel, ylim) in enumerate(zip(vars, ylabels, ylims)):
        axs[i, 1].scatter(np.arange(5), [var[animal]["20"], var[animal]["10"], var[animal]["2"], var[animal]["rev10"], var[animal]["rev20"]], color=rat_markers[animal][0], label=animal, marker=rat_markers[animal][1])
        axs[i, 1].set_title(f"")
        axs[i, 1].set_xticks(np.arange(5))
        axs[i, 1].set_xticklabels(["20", "10", "0", "-10", "-20"])
        axs[i, 1].set_ylabel(ylabel)
        axs[i, 1].set_xlim(-.5, 4.5)
        axs[i, 1].set_ylim(ylim)
        axs[i, 1].axhline(0, color='black', linestyle='--', linewidth=1)
    
axs[i, 0].set_xlabel("Distance")
axs[i, 1].set_xlabel(r'$v_{belt}$')
axs[i, 2].set_xlabel("All conditions pooled")

#mean per condition
for idx, cond in enumerate(["60", "90", "120"]):
    for jdx, var in enumerate([ mu, mu_t, mu_R, sigma, sigma_t, sigma_R,]):
        d = [var[animal][cond] for animal in animalList]
        mean, std = np.mean(d), np.std(d)/np.sqrt(len(d))
        s, p = stats.wilcoxon(d)
        if p < .05: axs[jdx, 0].scatter(idx+.3, mean, color='k', marker=r'$\ast$')
        axs[jdx, 0].errorbar(idx+.2, mean, yerr=std, color='black', marker='o', markersize=5, capsize=5, capthick=2, linewidth=2)

for idx, cond in enumerate(["20", "10", "2", "rev10", "rev20"]):
    for jdx, var in enumerate([ mu, mu_t, mu_R, sigma, sigma_t, sigma_R,]):
        d = [var[animal][cond] for animal in animalList]
        mean, std = np.mean(d), np.std(d)/np.sqrt(len(d))
        s, p = stats.wilcoxon(d)
        if p < .05: axs[jdx, 1].scatter(idx+.3, mean, color='k', marker=r'$\ast$')
        axs[jdx, 1].errorbar(idx+.2, mean, yerr=std, color='black', marker='o', markersize=5, capsize=5, capthick=2, linewidth=2)

test_all_conds_between_themselves(["60", "90", "120"], vars, ax=axs[:, 0])
test_all_conds_between_themselves(["20", "10", "2", "rev10", "rev20"], vars, ax=axs[:, 1])


Zmu = {animal: {key: (mu[animal][key] - np.mean([mu[animal][key] for animal in animalList]))/np.std([mu[animal][key] for animal in animalList]) for key in mu[animal]} for animal in animalList}
Zmu_t = {animal: {key: (mu_t[animal][key] - np.mean([mu_t[animal][key] for animal in animalList]))/np.std([mu_t[animal][key] for animal in animalList]) for key in mu_t[animal]} for animal in animalList}
Zmu_R = {animal: {key: (mu_R[animal][key] - np.mean([mu_R[animal][key] for animal in animalList]))/np.std([mu_R[animal][key] for animal in animalList]) for key in mu_R[animal]} for animal in animalList}
Zsigma = {animal: {key: (sigma[animal][key] - np.mean([sigma[animal][key] for animal in animalList]))/np.std([sigma[animal][key] for animal in animalList]) for key in sigma[animal]} for animal in animalList}
Zsigma_t = {animal: {key: (sigma_t[animal][key] - np.mean([sigma_t[animal][key] for animal in animalList]))/np.std([sigma_t[animal][key] for animal in animalList]) for key in sigma_t[animal]} for animal in animalList}
Zsigma_R = {animal: {key: (sigma_R[animal][key] - np.mean([sigma_R[animal][key] for animal in animalList]))/np.std([sigma_R[animal][key] for animal in animalList]) for key in sigma_R[animal]} for animal in animalList}

traits = {animal: [] for animal in animalList}
Zvars = [Zmu, Zmu_t, Zmu_R, Zsigma, Zsigma_t, Zsigma_R]
for j, zvar in enumerate(Zvars):
    for animal in animalList:
        zscores = [zvar[animal][cond] for cond in ["60", "90", "120", "20", "10", "2", "rev10", "rev20"]]
        pdf = stats.norm.pdf(np.linspace(-3, 3, 600), np.mean(zscores), np.std(zscores))
        traits[animal].append(np.mean(zscores))
        axs[j, 2].plot(pdf, np.linspace(-3, 3, 600), color=rat_markers[animal][0], linestyle=rat_markers[animal][2])
        Ri = compute_Ri(zvar, animalList)
        axs[j, 2].annotate(f'Ri = {Ri:.2f}: {interpret_Ri(Ri)}', xy=(0.05, 0.9), xycoords='axes fraction')
        axs[j, 2].set_ylim(-3, 3)
        axs[j, 2].set_ylabel("Z-scored " + ylabels[j])
        axs[j, 2].set_xlabel("")
        axs[j, 2].set_xticks([])

# pickle.dump([mu, mu_t, mu_R, sigma, sigma_t, sigma_R, loss], open("picklejar/running_time_model_parameters_fit.p", "wb"))

In [ ]:
fig, axs = plt.subplots(6, 2, figsize=(10, 20), gridspec_kw={'width_ratios': [3, 1]})
for j, zvar in enumerate(Zvars):
    for animal in animalList:
        zscores = [zvar[animal][cond] for cond in ["60", "90", "120", "20", "10", "2", "rev10", "rev20"]]

        axs[j, 0].plot(np.arange(8), zscores, color=rat_markers[animal][0], linestyle=rat_markers[animal][2])


        # axs[j, 1].hist(zscores, bins=np.linspace(-3, 3, 61), color=rat_markers[animal][0], alpha=0.25)
        Ri = compute_Ri(zvar, animalList)
        pdf = stats.norm.pdf(np.linspace(-3, 3, 600), np.mean(zscores), np.std(zscores))
        axs[j, 1].plot(pdf, np.linspace(-3, 3, 600), color=rat_markers[animal][0], linestyle=rat_markers[animal][2])
        axs[j, 1].annotate(f'Ri = {Ri:.2f}: {interpret_Ri(Ri)}', xy=(0.05, 0.9), xycoords='axes fraction')

In [ ]:
weights = {'RatF00': 212.02, 'RatF01': 205.85, 'RatF02': 193.75,
            'RatM00': 259.37, 'RatM01': 278.12, 'RatM02': 253.19,
            'RatF20': 220.10, 'RatF21': 215.53, 'RatF22': 215.0,
            'RatM20': 254.68, 'RatM21': 307.29, 'RatM22': 330.53,
            'RatF30': 217.32, 'RatF31': 228.95, 'RatF32': 216.80, 'RatF33': 222.77,
            'RatM30': 261.38, 'RatM31': 300.55, 'RatM32': 279.23, 
            'RatF30L': 217.32, 'RatF31L': 228.95, 'RatF32L': 216.80, 'RatF33L': 222.77,
            'RatM30L': 261.38, 'RatM31L': 300.55, 'RatM32L': 279.23}

fig, ax = plt.subplots(1, 6, figsize=(15, 3))
for j, zvar in enumerate(Zvars):
    _x, _y = [], []
    for animal in animalList:
        zscores = [zvar[animal][cond] for cond in ["60", "90", "120", "20", "10", "2", "rev10", "rev20"]]
        y = np.mean(zscores)
        x = weights[animal]

        ax[j].scatter(x, y, color=rat_markers[animal][0], marker=rat_markers[animal][1], s=100)
        ax[j].set_xlabel("Weight (g)")
        ax[j].set_ylabel(ylabels[j])
        ax[j].set_ylim(-2, 2)

        _x.append(x)
        _y.append(y)

    print(stats.pearsonr(_x, _y))
    gradient, intercept, r_value, p_value, std_err = stats.linregress(_x, _y)
    ax[j].plot(np.linspace(np.min(_x), np.max(_x), 100), gradient * np.linspace(np.min(_x), np.max(_x), 100) + intercept, color='black', lw=2)

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5), subplot_kw={'projection': 'polar'})

for idx, animal in enumerate(animalList):
        make_spider(axs, traits[animal], title='', 
        color=rat_markers[animal][0], marker=rat_markers[animal][1], linestyle=rat_markers[animal][2],
        labels=[r'$\mu_0$', r"$\mu_t$", r"$\mu_R$", r'$\sigma_0$', r"$\sigma_t$", r"$\sigma_R$"])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact
import numpy as np

def f(a=1, b=1, m=1, d=1):
    T = np.linspace(0, 2.5, 100)
    timecost = a*m*T
    plt.plot(T, timecost, color='g')

    speedcost = (b*m*d*d)/T
    plt.plot(T, speedcost, color='r')

    E_w = timecost + speedcost
    plt.plot(T, E_w, color='b')

    plt.ylim(0, 10)
    plt.show()

interact(f, a=(0, 10, 0.1), b=(0, 10, 0.1), m=(0, 10, 0.1), d=(0, 10, 0.1));